In [3]:
import os
import pandas as pd
import numpy as np
import pickle

# Setup

In [4]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://shai-nlp-course.netlify.app/clean-tweets.tsv"

In [5]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,أؤمن بأن الانسان ينطفئ جماله ابتعاد يحب بريق ا...,اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياة فنحن نعيش أرض أعدت للبلاء و...,خلو ضغط حية فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كآنت نقيه,نصم تسر حيت يرم لنس تعد كآن نقه


In [6]:
tweets = raw["clean_stemmed"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [7]:
sample = tweets.iloc[0]
sample

'اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذبل طفئ تحل ربع خرف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [8]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str]):
        self.vocab = self._create_vocab(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [9]:
sample_tokens = sample.split(" ")
sample_tokens

['اؤم',
 'بأن',
 'انس',
 'طفئ',
 'جمل',
 'بعد',
 'يحب',
 'برق',
 'عين',
 'خفي',
 'صبح',
 'ذبل',
 'طفئ',
 'تحل',
 'ربع',
 'خرف']

In [10]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

7754 61656


In [11]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'أجل': 0,
 'ظفر': 1,
 'رئع': 2,
 'أهداف': 3,
 'كئن': 4,
 'ركي': 5,
 'نحس': 6,
 'لعن': 7,
 'وآت': 8,
 'قم': 9,
 'أقسم': 10,
 'سءل': 11,
 'الك': 12,
 'يما': 13,
 'يرى': 14,
 'نخل': 15,
 'ورؤ': 16,
 'رتد': 17,
 'تآخ': 18,
 '❤': 19,
 'جبه': 20,
 'جندل': 21,
 'زدي': 22,
 'حكم': 23,
 'بعص': 24,
 'بيض': 25,
 'ابي': 26,
 'همو': 27,
 'تول': 28,
 'فجع': 29,
 'شغل': 30,
 'ولء': 31,
 'واز': 32,
 'خوهما…': 33,
 'شرف': 34,
 'دجل': 35,
 'عبدالعزيز': 36,
 'يضء': 37,
 'ريء': 38,
 'رسم': 39,
 'سفح': 40,
 'وباحتجاز': 41,
 'تحز': 42,
 'ايب': 43,
 '💪': 44,
 'مهم': 45,
 'نضم': 46,
 'للت': 47,
 'دار': 48,
 'يمي': 49,
 'نهى': 50,
 'قذر': 51,
 'بهئ': 52,
 'نفق': 53,
 'جاك': 54,
 'لأر': 55,
 'عالكل': 56,
 '“ورق”': 57,
 'معي': 58,
 'رتغال': 59,
 'قتصبو': 60,
 'تلم': 61,
 'رفي': 62,
 'حللو': 63,
 'رمج': 64,
 'نهز': 65,
 'كصت': 66,
 'احن': 67,
 'طغـ': 68,
 'نبج': 69,
 'ؤخذ': 70,
 'اشد': 71,
 'لذذ': 72,
 'سفت': 73,
 'ستف': 74,
 'مستوطنةصهيونية': 75,
 'أزم': 76,
 'نفح': 77,
 'استر': 78,
 'يتأ': 79,
 'اليه': 80,
 'إ

In [16]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str]):
        super().__init__(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        all_word_tokens = set([token for sample in corpus for token in sample.split(" ")])
        vocab = {token: index for index, token in enumerate(word_level_tokens, start=1)} 
        vocab["[PAD]"] = 0
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [17]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets)

## Character Level Tokenization

In [18]:
sample_tokens = list(sample)
sample_tokens

['ا',
 'ؤ',
 'م',
 ' ',
 'ب',
 'أ',
 'ن',
 ' ',
 'ا',
 'ن',
 'س',
 ' ',
 'ط',
 'ف',
 'ئ',
 ' ',
 'ج',
 'م',
 'ل',
 ' ',
 'ب',
 'ع',
 'د',
 ' ',
 'ي',
 'ح',
 'ب',
 ' ',
 'ب',
 'ر',
 'ق',
 ' ',
 'ع',
 'ي',
 'ن',
 ' ',
 'خ',
 'ف',
 'ي',
 ' ',
 'ص',
 'ب',
 'ح',
 ' ',
 'ذ',
 'ب',
 'ل',
 ' ',
 'ط',
 'ف',
 'ئ',
 ' ',
 'ت',
 'ح',
 'ل',
 ' ',
 'ر',
 'ب',
 'ع',
 ' ',
 'خ',
 'ر',
 'ف']

In [19]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

346


In [20]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'🤨': 0,
 '🤬': 1,
 'ط': 2,
 '🙈': 3,
 '💭': 4,
 '😜': 5,
 '🇬': 6,
 'ﺬ': 7,
 'ﻬ': 8,
 'ا': 9,
 '🖕': 10,
 '📺': 11,
 '😌': 12,
 '🌚': 13,
 '\u200d': 14,
 '🖤': 15,
 '⚽': 16,
 '❤': 17,
 '😓': 18,
 'ﻳ': 19,
 '💘': 20,
 '✈': 21,
 '🔞': 22,
 '🦋': 23,
 '🙌': 24,
 'ك': 25,
 '⌛': 26,
 'ـ': 27,
 '😨': 28,
 'ڪ': 29,
 'ﻷ': 30,
 '🐐': 31,
 '😱': 32,
 '🇱': 33,
 '💪': 34,
 'ﻨ': 35,
 '🏴': 36,
 'ص': 37,
 'ม': 38,
 'ح': 39,
 'ﺱ': 40,
 '☝': 41,
 '😃': 42,
 'ﺃ': 43,
 '🏼': 44,
 '🌿': 45,
 '💖': 46,
 '💕': 47,
 '😞': 48,
 '👇': 49,
 '✔': 50,
 'ﺍ': 51,
 'غ': 52,
 '😙': 53,
 '࿐': 54,
 'ﻲ': 55,
 'ئ': 56,
 '🤷': 57,
 '🌼': 58,
 '😐': 59,
 '🔥': 60,
 '😢': 61,
 '⁰': 62,
 '😟': 63,
 '😻': 64,
 'أ': 65,
 '👌': 66,
 '🌹': 67,
 'ﺴ': 68,
 '💳': 69,
 'ﻥ': 70,
 '💤': 71,
 '✿': 72,
 '🇧': 73,
 '\u200c': 74,
 '💙': 75,
 '🇸': 76,
 '🍷': 77,
 'ﺙ': 78,
 '❀': 79,
 '🦅': 80,
 'ﻖ': 81,
 '🌸': 82,
 '😳': 83,
 '🇲': 84,
 '💡': 85,
 '🏻': 86,
 'ﻃ': 87,
 '😔': 88,
 '“': 89,
 'ھ': 90,
 '💰': 91,
 'ﺀ': 92,
 'ﻐ': 93,
 '☹': 94,
 '💐': 95,
 '﴾': 96,
 '💞': 97,
 '😊': 98,
 '❣': 99,


In [21]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document 
class CharacterLevelTokenizer(Tokenizer):
    ...

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [22]:
tokenized_tweets = word_level_tokenizer.tokenize(tweets)

In [23]:
max([len(t) for t in tokenized_tweets])

25

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [24]:
word_level_tokenizer_path = os.path.join(data_dir, "word_tokenizer.pkl")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)


In [85]:
# TODO: Pickle character level tokenizer